In [1]:
import ctypes
from ctypes import byref
clib = ctypes.CDLL("../fortran/wrapper.so")
clib.flux.restype = None

In [2]:
def flux_numerical(u1, u2, rp, rm, bp, bpm, theta, nn):
    
    xp, yp = -bp, 0.0
    xm, ym = -bp + bpm * np.cos(theta), bpm * np.sin(theta)
    
    ld = lambda p: [
        1 - u1 * (1 - np.sqrt(1 - p[0]**2 - p[1]**2)) 
        - u2 * (1 - np.sqrt(1 - p[0]**2 - p[1]**2))**2 
        for p in p
    ]
    ld1 = 1 - u1 - 2 * u2
    ld2 = u1 + 2 * u2
    ld3 = u2
    
    norm = ld1 * np.pi + ld2 * 2 * np.pi / 3 + ld3 * np.pi / 2
    rp2 = rp * rp
    rm2 = rm * rm
    bpm = np.sqrt((xp - xm)**2 + (yp - ym)**2)
    
    res = nn ** 2

    if bpm < (rp + rm):
        
        if bpm + rm > rp:
            scale = 2 * (bpm + rm) + rm / 10
        else: 
            scale = 2 * rp + rm / 10
        
        n = int(np.sqrt(res * (4 * rp) ** 2))
        rat = np.sqrt(3) / 2
        nx = int(np.sqrt(n / rat))
        ny = n // nx
        xv, yv = np.meshgrid(np.arange(nx), np.arange(ny))
        xv = xv * rat
        xv[::2, :] += rat / 2
        xv = xv / len(xv) * scale + xp - scale / 2
        yv = yv / len(yv) * scale + yp - scale / 2
        points = np.concatenate((xv.flatten()[:, None], yv.flatten()[:, None]), axis=1)
    else:
        lxp = xp - rp
        uxp = xp + rp
        lxm = xm - rm
        uxm = xm + rm
    
        lyp = yp - rp
        uyp = yp + rp
        lym = ym - rm
        uym = ym + rm
    
        nump = int(np.sqrt(res * (2 * rp) ** 2))
        rat = np.sqrt(3) / 2
        nxp = int(np.sqrt(nump / rat))
        nyp = nump // nxp
        xvp, yvp = np.meshgrid(np.arange(nxp), np.arange(nyp))
        xvp = xvp * ratio
        xvp[::2, :] += rat / 2
        xvp = xvp / len(xvp) * 2 * rp + xp - rp
        yvp = yvp / len(yvp) * 2 * rp + yp - rp
        pointsp = np.concatenate((xvp.flatten()[:, None], yvp.flatten()[:, None]), axis=1)
        
        numm = int(np.sqrt(res * (2 * rm) ** 2))
        rat = np.sqrt(3) / 2
        nxm = int(np.sqrt(numm / rat))
        nym = numm // nxm
        xvm, yvm = np.meshgrid(np.arange(nxm), np.arange(nym))
        xvm = xvm * ratio
        xvm[::2, :] += rat / 2
        xvm = xvm / len(xvm) * 2 * rm + xm - rm
        yvm = yvm / len(yvm) * 2 * rm + ym - rm
        pointsm = np.concatenate((xvm.flatten()[:, None], yvm.flatten()[:, None]), axis=1)
        points = np.concatenate((pointsp, pointsm))
    
    dx = points[1, 0] - points[0, 0]
    dd =  dx * dx * 2 / np.sqrt(3)
    dp2 = (points[:, 0] - xp)**2 + (points[:, 1] - yp)**2
    dm2 = (points[:, 0] - xm)**2 + (points[:, 1] - ym)**2
    ds2 = points[:, 0]**2 + points[:, 1]**2
    
    occulted = ((dp2 < rp2) | (dm2 < rm2)) & (ds2 < 1)
    return - np.sum(ld(points[occulted])) * dd / norm

In [3]:
bp = 0.4
bpm = 0.05
theta = np.pi / 4

xp, yp = -bp, 0.0
xm, ym = -bp + bpm * np.cos(theta), bpm * np.sin(theta)

In [9]:
import sys
sys.path.append('../gefera')
import testing

u1 = 0.5
u2 = 0.3
rp = 0.2
rm = 0.17

#testing.flux_numerical(u1, u2, rp, rm, xp, yp, xm, ym, 100000)

In [10]:
from phot import flux
flux(u1, u2, rp, rm, np.array([bp]), np.array([bpm]), np.array([np.cos(theta)]), np.array([np.sin(theta)]))[0][0]

-0.050227017987842015

In [13]:
bp = 1.5
bpm = 0.4
theta = 0.3

xp, yp = -bp, 0.0
xm, ym = -bp + bpm * np.cos(theta), bpm * np.sin(theta)

rp = 0.3
rm = 0.2

print(rp, rm, bp, bpm, theta)

t = np.linspace(0, 2 * np.pi, 1000)
plt.figure(figsize=(5, 5))
plt.xlim(-1.5, 1.5)
plt.ylim(-1.5, 1.5)
plt.plot(np.cos(t), np.sin(t))
plt.plot(xp + rp * np.cos(t), yp + rp * np.sin(t))
plt.plot(xm + rm * np.cos(t), ym + rm * np.sin(t))

0.3 0.2 1.5 0.4 0.3


NameError: name 'plt' is not defined

In [14]:
testing.change_coords(xp, yp, xm, ym)

(1.5, 0.4, 0.2999999999999996)

In [22]:
coords = [
        [0.95, 1, 1.15, 0.75],
        [0.95, 1, 0.7, 0.75],
        [0.8, 0.8, 1.15, 0.75],
        [0.4, 0.4, 0.4, 0.4],
        [0.4, 0.4, 0.6, 0.2],
        [0.4, 0.4, 0.65, 0.65],
        [0.55, 0.55, 0.52, 0.52],
        [0.6, 0.6, 0.4, 0.4],
        [0.65, 0.65, 0.65, 0.65],
        [0.6, 0.6, 0.75, 0.75],
        [0.8, 0.8, 0.65, 0.65],
        [0.52, 0.52, 0.7, 0.7],
        [0.88, 0.88, 0.7, 0.7],
        [0.85, 0.9, 1.05, 0.5],
        [0.5, 0.55, 0.8, 0.3],
        [0.6, 0.65, 0.9, 0.4]
    ]

In [25]:
for xp, yp, xm, ym in coords:
    print(xp, yp)

0.95 1
0.95 1
0.8 0.8
0.4 0.4
0.4 0.4
0.4 0.4
0.55 0.55
0.6 0.6
0.65 0.65
0.6 0.6
0.8 0.8
0.52 0.52
0.88 0.88
0.85 0.9
0.5 0.55
0.6 0.65
